In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [ ]:
#read in the data
mid=pd.read_csv("sample out mid.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#add elapsed time columns
timestamps1=pd.to_datetime(mid["mass 2.0_time"],format="%Y/%m/%d %H:%M:%S")
timestamps2=pd.to_datetime(mid["mass 28.0_time"],format="%Y/%m/%d %H:%M:%S")
timestamps3=pd.to_datetime(mid["mass 44.0_time"],format="%Y/%m/%d %H:%M:%S")

runtime1=(timestamps1-timestamps1[0]).dt.total_seconds()    
mid.insert(1,"Elapsed time, mass 2",runtime1)
runtime2=(timestamps2-timestamps2[0]).dt.total_seconds()    
mid.insert(10,"Elapsed time, mass 28",runtime2)
runtime3=(timestamps3-timestamps2[0]).dt.total_seconds()    
mid.insert(16,"Elapsed time, mass 44",runtime3)
mid.head()


In [ ]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

In [ ]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



### Position of the EGA

 

<div>
<img src="20221117_142845-Copy1.jpg" width="1000"/>
</div>

### Data processing

In [ ]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(mid['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='h2')
plt.plot(mid['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='co')
plt.plot(mid['Elapsed time, mass 44'],mid["mass 44.0_value"],marker=".", markersize=4,label='co2')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("RGA scans overlay")
plt.yscale('log')


In [ ]:
#Filtering background signal

#Find the timestamp where emission was turned on
#print("Result 1:",hv.loc[hv["Comment"]=="emission start"])
x=pd.Timestamp(2022, 11, 10, 17,12, 45)

#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#average the signal before this timestamp and subtract it from the Em-on part
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 1:",mid["mass 2, corr"].describe())


#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][102:195]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:102]/h2_sens
base_h2.describe()

In [ ]:
#CO

#Find the timestamp where emission was turned on
#print("Result 1:",hv.loc[hv["Comment"]=="emission start"])
x=pd.Timestamp(2022, 11, 10, 17,12, 45)

#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 28.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#average this signal and subtract it from the Em-on part
bck=np.mean(mid["mass 28.0_value"][0:102])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 28, corr"]=mid["mass 28.0_value"][102:]-bck
mid["mass 28, corr"].describe()

#getting the differential spectrum
#first get the timestamp where grid voltage was changed to -500V
print("Location:",hv.loc[hv["Comment"]=="grid repelling"])
y=pd.Timestamp(2022, 11, 10, 17,22, 56)

#then find the time value closest to it in the MID scan file
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 28.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#subtract grid repelling from grid transparent
#getting the mean signal when grid is repelling
grid2=np.mean(mid["mass 28, corr"][195:])
mid["sample sig, mass 28"]=mid["mass 28, corr"][102:195]-grid2
mid["sample sig, mass 28"].describe()


### Plotting

In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current recording")
plt.yscale('log')


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2 pressure reading')
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1 pressure reading')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Barion 2 pressure evolution")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [ ]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 296
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
I_em = np.mean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


In [ ]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)